In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install streamlit-searchbox
#!pip install wikipedia-api
#!pip install wikipedia

In [ ]:
!pip install streamlit
!npm install localtunnel

In [ ]:
!pip install transformers
!pip install pytorch_lightning

In [ ]:
%%writefile app.py

import streamlit as st
from typing import List
import pandas as pd
import sqlite3
from transformers import AutoModelWithLMHead, AutoTokenizer

st.set_page_config(page_title="TEAM 1: LEWINSKY", page_icon="😷", layout="wide")

conn = sqlite3.connect('/kaggle/input/airline-dataset/travel.sqlite')

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")



# CSS styles
css_styles = """
<style>
    .card {
        background: #BAD2DE;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.4);
        border-radius: 5px;
        margin: 50px 20px 20px 20px;
        padding: 20px;
        text-align: center;
        color: black;
        float: left;
    }

    .card:first-child {
        margin-top: 20px;
    }

    .card:nth-child(2) {
        margin: 10px 0;
        width: 15%;
        padding: 20px;
    }


    .title {
        font-size: 25px;
    }

    .icon {
        margin: 50px 0;

        svg {
            width: 100px;
            height: 100px;

            path {
                fill: #F6B352;
            }
        }
    }

    .features ul {
        padding: 0;
        margin: 20px 0 50px 0;
        list-style-type: none;
    }

    .features li {
        margin: 10px 0;
        font-size: 14px;

        span {
            border-bottom: 2px dotted #F6B352;
        }
    }

    .btn {
        display: block;
        background: #F6B352;
        color: black;
        padding: 15px 20px;
        margin: 20px 0;
        border-radius: 5px;
        box-shadow: rgba(0, 0, 0, 0.9);
        transition: all 200ms ease-in-out;
        text-decoration: none;
    }

    .btn:hover {
        background: #F68657;
    }
</style>
"""

# Display CSS styles
st.markdown(css_styles, unsafe_allow_html=True)

# Display cards
st.markdown(
    """
    <div class="card">
        <div class="title">Link Database</div>
        <div class="icon">
            <svg enable-background="new 0 0 512 512" height="512px" id="Layer_1" version="1.1" viewBox="0 0 512 512"
                width="512px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg"
                xmlns:xlink="http://www.w3.org/1999/xlink">
                <path
                    d="M468.493,101.637L371.955,5.098H159.57v77.231H43.724v424.769h308.923v-77.231h115.846V101.637z   M371.955,32.401l69.236,69.235h-69.236V32.401z M63.031,487.79V101.637h173.769v96.538h96.538V487.79H63.031z M256.108,109.632  l69.236,69.235h-69.236V109.632z M352.647,410.56V178.867l-96.538-96.538h-77.231V24.406h173.769v96.538h96.538V410.56H352.647z"
                    fill="#37404D" />
            </svg>
        </div>
        <div class="features">
            <ul>
                <li><span>Link</span> Tables</li>
                <li><span>1GB</span> Storage</li>
                <li><span>Link</span> Rows</li>
                <li><span>Join</span> Columns</li>
            </ul>
        </div>
        <a href="#" class="btn">SELECT</a>
    </div>

    <div class="card">
        <div class="title">Recent Searches</div>
        <div class="icon">
            <svg enable-background="new 0 0 512 512" height="512px" id="Layer_1" version="1.1" viewBox="0 0 512 512"
                width="512px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg"
                xmlns:xlink="http://www.w3.org/1999/xlink">
                <path
                    d="M285.928,113.067c62.492,0,113.327,50.827,113.327,113.327c0,0.344-0.041,0.705-0.066,1.049  c-0.049,0.836-0.107,1.672-0.123,2.525l-0.426,17.133l17.159,0.065c41.53,0.115,75.313,34.005,75.313,75.535  c0,41.415-33.718,75.305-75.157,75.518l-3.664,0.016H104.977c-46.244-0.049-83.872-37.693-83.872-83.929  c0-35.825,22.806-67.714,56.737-79.356l9.444-3.229l1.664-9.838c4.115-24.282,24.97-41.907,49.588-41.907  c7.846,0,15.428,1.82,22.536,5.394l15.306,7.689l7.386-15.444C202.531,138.398,242.635,113.067,285.928,113.067 M285.928,96.277  c-51.778,0-96.358,30.315-117.303,74.092c-9.059-4.558-19.256-7.182-30.086-7.182c-33.233,0-60.762,24.184-66.14,55.893  C32.82,232.657,4.316,270.104,4.316,314.307c0,55.597,45.063,100.669,100.644,100.718h311.084v-0.016  c50.777-0.263,91.856-41.481,91.856-92.308c0-50.909-41.177-92.177-92.053-92.324c0.033-1.344,0.197-2.639,0.197-3.984  C416.044,154.532,357.79,96.277,285.928,96.277L285.928,96.277z" fill="#37404D" />
            </svg>
        </div>
        <div class="features">
            <ul>
                <li><span>Show</span> all flights</li>
                <li><span>Show</span> all bookings</li>
                <li><span>Show</span> tickets</li>
                <li><span>Find</span> aircraft data </li>
            </ul>
        </div>
        <a href="#" class="btn">SELECT</a>
    </div>

    <div class="card">
        <div class="title">Search using SQL</div>
        <div class="icon">
            <svg enable-background="new 0 0 512 512" height="512px" id="Layer_1" version="1.1" viewBox="0 0 512 512"
                width="512px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg"
                xmlns:xlink="http://www.w3.org/1999/xlink">
                <path
                    d="M316.01,199.02L256.134,14.817L196.239,199.02H1.134l158.102,113.324L98.53,496.487l157.604-114.232  l157.585,114.232l-60.687-184.143L511.134,199.02H316.01z M335.084,318.257l42.407,128.63L267.22,366.963l-11.086-8.033  l-11.086,8.033l-110.291,79.923l42.408-128.63l4.353-13.18l-11.289-8.08L59.903,217.909h136.336h13.724l4.242-13.051l41.929-128.957  l41.91,128.957l4.242,13.051h13.724h136.336l-110.327,79.088l-11.27,8.08L335.084,318.257z" fill="#37404D" />
            </svg>
        </div>
        <div class="features">
            <ul>
                <li><span>SELECT aircraft_code, model FROM aircrafts_data;</span> Edits</li>
                <li><span>1</span> 1</li>
                <li><span>Recent</span> SELECT airport_code, airport_name FROM airports_data;</li>
                <li><span>X</span> SELECT * FROM flights;</li>
            </ul>
        </div>
        <a href="#" class="btn">SELECT</a>
    </div>
    """,
    unsafe_allow_html=True
)

def get_sql(query):
    input_text = "translate English to SQL: %s </s>" % query
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                             attention_mask=features['attention_mask'])
  
    return tokenizer.decode(output[0])

queries = [
    ("Find bookings made on a specific date", "SELECT book_ref, total_amount FROM bookings WHERE book_date = '[specific_date]';"),
    ("Select all aircrafts and their models", "SELECT aircraft_code, model FROM aircrafts_data;"),
    ("Select airports with their names and cities", "SELECT airport_code, airport_name->>'name' AS airport_name, city->>'name' AS city FROM airports_data;"),
    # Add more queries as needed
]

def search(searchterm: str) -> List[str]:
    # Filter queries based on searchterm
    result = [query[0] for query in queries if searchterm.lower() in query[0].lower()]
    return result

# Streamlit app with custom search box
def main():
    user_query = st.text_input("Enter your search query:")

    ai_result = get_sql(user_query)
    ai_result = ai_result.replace('<pad>', '').replace('</s>', ';').strip()
    st.subheader("AI Results:")
    st.text(ai_result)

    if st.button("Execute SQL"):
        cursor = conn.cursor()
        sql_result = cursor.execute(user_query).fetchall()
        st.subheader("SQL Query Results:")
        st.write(sql_result)
    

    # CSS styles
    css_styles2 = """
    <style>
        .ui {
            display: flex;
            justify-content: space-around;
            list-style: none;
            padding: 0;
        }

        .card {
            background: #BAD2DE;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.4);
            border-radius: 5px;
            margin: 20px;
            padding: 20px;
            text-align: center;
            color: black;
            width: calc(33.3333% - 40px);
        }

        .card p {
            margin: 10px 0;
            font-size: 14px;
        }
    </style>
    """

    # Display CSS styles
    st.markdown(css_styles2, unsafe_allow_html=True)

    # Sample data
    if st.button("Test"):
        cursor = conn.cursor()
        sql_test = cursor.execute("SELECT aircraft_code, model FROM aircrafts_data;").fetchall()

        st.subheader("SQL Test")

        # Display SQL test results in the card structure
        st.markdown("<ul class='ui'>", unsafe_allow_html=True)
        for row in sql_test:
            st.markdown(f"""
                <li class='card'>
                    {''.join([f"<p>{item}</p>" for item in row])}
                </li>
            """, unsafe_allow_html=True)
        st.markdown("</ul>", unsafe_allow_html=True)

        conn.close()

if __name__ == '__main__':
    main()

In [ ]:
%%writefile app.py

import streamlit as st
from typing import List
import pandas as pd
import sqlite3
from transformers import AutoModelWithLMHead, AutoTokenizer

st.set_page_config(page_title="TEAM 1: LEWINSKY", page_icon="😷", layout="wide")

conn = sqlite3.connect('/kaggle/input/airline-dataset/travel.sqlite')

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

import streamlit as st
import pandas as pd
import io
import requests


url="/kaggle/input/parttwo/data.csv"

df=pd.read_csv(url)

df['CREATED'] = pd.to_datetime(df['CREATED'])
df['LAST_ALTERED'] = pd.to_datetime(df['LAST_ALTERED'])

df2 = df


def human_bytes(B):
    """Return the given bytes as a human friendly KB, MB, GB, or TB string."""
    B = float(B)
    KB = float(1024)
    MB = float(KB ** 2)  # 1,048,576
    GB = float(KB ** 3)  # 1,073,741,824
    TB = float(KB ** 4)  # 1,099,511,627,776

    if B < KB:
        return '{0} {1}'.format(B, '')
    elif KB <= B < MB:
        return '{0:.2f}'.format(B / KB)
    elif MB <= B < GB:
        return '{0:.2f}'.format(B / MB)
    elif GB <= B < TB:
        return '{0:.2f}'.format(B / GB)
    elif TB <= B:
        return '{0:.2f}'.format(B / TB)


def human_bytes_text(B):
    """Return the given bytes as a human friendly KB, MB, GB, or TB string."""
    B = float(B)
    KB = float(1024)
    MB = float(KB ** 2)  # 1,048,576
    GB = float(KB ** 3)  # 1,073,741,824
    TB = float(KB ** 4)  # 1,099,511,627,776

    if B < KB:
        return 'Bytes'
    elif KB <= B < MB:
        return 'KB'
    elif MB <= B < GB:
        return 'MB'
    elif GB <= B < TB:
        return 'GB'
    elif TB <= B:
        return 'TB'


def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return ('%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])).replace('.00', '')


def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)


def remote_css(url):
    st.markdown(f'<link href="{url}" rel="stylesheet">',
                unsafe_allow_html=True)


def header_bg(table_type):
    if table_type == "BASE TABLE":
        return "tablebackground"
    elif table_type == "VIEW":
        return "viewbackground"
    else:
        return "mvbackground"


remote_css(
    "https://cdnjs.cloudflare.com/ajax/libs/semantic-ui/2.4.1/semantic.min.css")


local_css("/kaggle/input/cssfile/style5.css")
cb_view_details = st.sidebar.checkbox('View Details')

if cb_view_details:
    view_details=""
else:
    view_details="""style="display: none;" """

selectbox_orderby = st.sidebar.selectbox("Order By", ('A → Z', 'Z → A', 'Data Size ↓', 'Data Size ↑',
                                         'Rows ↓', 'Rows ↑', 'Date Created ↓', 'Date Created ↑', 'Date Altered ↓', 'Date Altered ↑'))
#button_clicked = st.button("OK")

all_option = pd.Series(['All'], index=[9999999])

#TABLE_SCHEMA=TABLE_SCHEMA.append({'TABLE_SCHEMA':'All'},ignore_index = True)

if 'selectbox_database_key' not in st.session_state:
    st.session_state.selectbox_database_key = 10
    st.session_state.selectbox_schema_key = 20
    st.session_state.selectbox_owner_key = 30
    st.session_state.selectbox_table_type_key = 40
    st.session_state.selectbox_max_rows_key = 50
    st.session_state.selectbox_data_size_key = 60

# Table Catalog/Database
fv_database = df['TABLE_CATALOG'].drop_duplicates()
fv_database = pd.concat([fv_database,all_option])

selectbox_database = st.sidebar.selectbox(
    'Database', fv_database, index=len(fv_database)-1, key=st.session_state.selectbox_database_key)

if selectbox_database != 'All':
    df = df.loc[df['TABLE_CATALOG'] == selectbox_database]
else:
    df = df.loc[df['TABLE_CATALOG'].isin(fv_database)]

# Table Schema
fv_table_schema = df['TABLE_SCHEMA'].drop_duplicates()
fv_table_schema = pd.concat([fv_table_schema, all_option])

selectbox_schema = st.sidebar.selectbox(
    "Table Schema", fv_table_schema, len(fv_table_schema)-1, key=st.session_state.selectbox_schema_key)

if selectbox_schema != 'All':
    df = df.loc[df['TABLE_SCHEMA'] == selectbox_schema]
else:
    df = df.loc[df['TABLE_SCHEMA'].isin(fv_table_schema)]

# Table Owner
fv_owner = df['TABLE_OWNER'].drop_duplicates()
fv_owner = pd.concat([fv_owner,all_option])
selectbox_owner = st.sidebar.selectbox(
    "Table Owner", fv_owner, len(fv_owner)-1, key=st.session_state.selectbox_owner_key)

if selectbox_owner != 'All':
    df = df.loc[df['TABLE_OWNER'] == selectbox_owner]
else:
    df = df.loc[df['TABLE_OWNER'].isin(fv_owner)]

# Table Type
fv_table_type = df['TABLE_TYPE'].drop_duplicates()
selectbox_table_type = st.sidebar.multiselect(
    'Table Type', fv_table_type, fv_table_type, key=st.session_state.selectbox_table_type_key)

if len(selectbox_table_type) > 0:
    df = df.loc[df['TABLE_TYPE'].isin(selectbox_table_type)]
else:
    df = df.loc[df['TABLE_TYPE'].isin(fv_table_type)]

# #!!! This part is disabled since sliders are causing performance issues with large datasets.!!!
# # data size selection
max_data_mb = int(df['BYTES'].max()/1048576)
step_size = 1

if max_data_mb>1000:
    step_size=10
elif max_data_mb>1000000:
    step_size=100
elif max_data_mb>1000000000:
    step_size=1000
elif max_data_mb>1000000000000:
    step_size=10000      

data_size = st.sidebar.slider(
    'Data Size (MB)', 0, max_data_mb+1, (0, max_data_mb+1), key=st.session_state.selectbox_data_size_key, step=step_size)
df = df.loc[(df['BYTES'] >= data_size[0]*1048576) &
            (df['BYTES'] <= data_size[1]*1048576)]

# rows selection
max_rows = int(df['ROW_COUNT'].max())
step_size = 10

if max_rows>1000000:
    step_size=100
elif max_rows>1000000000:
    step_size=1000
elif max_rows>1000000000000:
    step_size=10000    

data_rows = st.sidebar.slider('Number of Rows', 0, max_rows+1,
                              (0, max_rows+1), key=st.session_state.selectbox_max_rows_key, step=step_size)
df = df.loc[(df['ROW_COUNT'] >= data_rows[0]) &
            (df['ROW_COUNT'] <= data_rows[1])]


def reset_button():
    st.session_state.selectbox_database_key = st.session_state.selectbox_database_key+1
    st.session_state.selectbox_schema_key = st.session_state.selectbox_schema_key+1
    st.session_state.selectbox_owner_key = st.session_state.selectbox_owner_key+1
    st.session_state.selectbox_table_type_key = st.session_state.selectbox_table_type_key+1
    st.session_state.selectbox_max_rows_key = st.session_state.selectbox_max_rows_key+1
    st.session_state.selectbox_data_size_key = st.session_state.selectbox_data_size_key+1


clear_button = st.sidebar.button(
    label='Clear Selections', on_click=reset_button)

if clear_button:
    df = df2

# Card order
orderby_column = ''
orderby_asc = True


if selectbox_orderby == 'A → Z':
    orderby_column = 'TABLE_NAME'
    orderby_asc = True
elif selectbox_orderby == 'Z → A':
    orderby_column = 'TABLE_NAME'
    orderby_asc = False
elif selectbox_orderby == 'Data Size ↓':
    orderby_column = 'BYTES'
    orderby_asc = False
elif selectbox_orderby == 'Data Size ↑':
    orderby_column = 'BYTES'
    orderby_asc = True
elif selectbox_orderby == 'Rows ↓':
    orderby_column = 'ROW_COUNT'
    orderby_asc = False
elif selectbox_orderby == 'Rows ↑':
    orderby_column = 'ROW_COUNT'
    orderby_asc = True
elif selectbox_orderby == 'Date Created ↓':
    orderby_column = 'CREATED'
    orderby_asc = False
elif selectbox_orderby == 'Date Created ↑':
    orderby_column = 'CREATED'
    orderby_asc = True
elif selectbox_orderby == 'Date Altered ↓':
    orderby_column = 'LAST_ALTERED'
    orderby_asc = False
elif selectbox_orderby == 'Date Altered ↑':
    orderby_column = 'LAST_ALTERED'
    orderby_asc = True


df.sort_values(by=[orderby_column], inplace=True, ascending=orderby_asc)



#here

# CSS styles
css_styles1 = """
<style>
    .card {
        background: #BAD2DE;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.4);
        border-radius: 5px;
        margin: 50px 20px 20px 20px;
        padding: 20px;
        text-align: center;
        color: black;
        float: left;
    }

    .card:first-child {
        margin-top: 20px;
    }

    .card:nth-child(2) {
        margin: 10px 0;
        width: 15%;
        padding: 20px;
    }


    .title {
        font-size: 25px;
    }

    .icon {
        margin: 50px 0;

        svg {
            width: 100px;
            height: 100px;

            path {
                fill: #CBE2DA;
            }
        }
    }

    .features ul {
        padding: 0;
        margin: 20px 0 50px 0;
        list-style-type: none;
    }

    .features li {
        margin: 10px 0;
        font-size: 14px;

        span {
            border-bottom: 2px dotted #CBE2DA;
        }
    }

    .btn {
        display: block;
        background: #CBE2DA;
        color: black;
        padding: 15px 20px;
        margin: 20px 0;
        border-radius: 5px;
        box-shadow: rgba(0, 0, 0, 0.9);
        transition: all 200ms ease-in-out;
        text-decoration: none;
    }

    .btn:hover {
        background: #CBE2DA;
    }
</style>
"""

# Display CSS styles
st.markdown(css_styles1, unsafe_allow_html=True)

# Display cards
st.markdown(
    """
    <div class="card">
        <div class="title">Link Database</div>
        <div class="icon">
            <svg enable-background="new 0 0 512 512" height="512px" id="Layer_1" version="1.1" viewBox="0 0 512 512"
                width="512px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg"
                xmlns:xlink="http://www.w3.org/1999/xlink">
                <path
                    d="M468.493,101.637L371.955,5.098H159.57v77.231H43.724v424.769h308.923v-77.231h115.846V101.637z   M371.955,32.401l69.236,69.235h-69.236V32.401z M63.031,487.79V101.637h173.769v96.538h96.538V487.79H63.031z M256.108,109.632  l69.236,69.235h-69.236V109.632z M352.647,410.56V178.867l-96.538-96.538h-77.231V24.406h173.769v96.538h96.538V410.56H352.647z"
                    fill="#37404D" />
            </svg>
        </div>
        <div class="features">
            <ul>
                <li><span>Link</span> Tables</li>
                <li><span>1GB</span> Storage</li>
                <li><span>Link</span> Rows</li>
                <li><span>Join</span> Columns</li>
            </ul>
        </div>
        <a href="#" class="btn">SELECT</a>
    </div>

    <div class="card">
        <div class="title">Recent Searches</div>
        <div class="icon">
            <svg enable-background="new 0 0 512 512" height="512px" id="Layer_1" version="1.1" viewBox="0 0 512 512"
                width="512px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg"
                xmlns:xlink="http://www.w3.org/1999/xlink">
                <path
                    d="M285.928,113.067c62.492,0,113.327,50.827,113.327,113.327c0,0.344-0.041,0.705-0.066,1.049  c-0.049,0.836-0.107,1.672-0.123,2.525l-0.426,17.133l17.159,0.065c41.53,0.115,75.313,34.005,75.313,75.535  c0,41.415-33.718,75.305-75.157,75.518l-3.664,0.016H104.977c-46.244-0.049-83.872-37.693-83.872-83.929  c0-35.825,22.806-67.714,56.737-79.356l9.444-3.229l1.664-9.838c4.115-24.282,24.97-41.907,49.588-41.907  c7.846,0,15.428,1.82,22.536,5.394l15.306,7.689l7.386-15.444C202.531,138.398,242.635,113.067,285.928,113.067 M285.928,96.277  c-51.778,0-96.358,30.315-117.303,74.092c-9.059-4.558-19.256-7.182-30.086-7.182c-33.233,0-60.762,24.184-66.14,55.893  C32.82,232.657,4.316,270.104,4.316,314.307c0,55.597,45.063,100.669,100.644,100.718h311.084v-0.016  c50.777-0.263,91.856-41.481,91.856-92.308c0-50.909-41.177-92.177-92.053-92.324c0.033-1.344,0.197-2.639,0.197-3.984  C416.044,154.532,357.79,96.277,285.928,96.277L285.928,96.277z" fill="#37404D" />
            </svg>
        </div>
        <div class="features">
            <ul>
                <li><span>Show</span> all flights</li>
                <li><span>Show</span> all bookings</li>
                <li><span>Show</span> tickets</li>
                <li><span>Find</span> aircraft data </li>
            </ul>
        </div>
        <a href="#" class="btn">SELECT</a>
    </div>

    <div class="card">
        <div class="title">Search using SQL</div>
        <div class="icon">
            <svg enable-background="new 0 0 512 512" height="512px" id="Layer_1" version="1.1" viewBox="0 0 512 512"
                width="512px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg"
                xmlns:xlink="http://www.w3.org/1999/xlink">
                <path
                    d="M316.01,199.02L256.134,14.817L196.239,199.02H1.134l158.102,113.324L98.53,496.487l157.604-114.232  l157.585,114.232l-60.687-184.143L511.134,199.02H316.01z M335.084,318.257l42.407,128.63L267.22,366.963l-11.086-8.033  l-11.086,8.033l-110.291,79.923l42.408-128.63l4.353-13.18l-11.289-8.08L59.903,217.909h136.336h13.724l4.242-13.051l41.929-128.957  l41.91,128.957l4.242,13.051h13.724h136.336l-110.327,79.088l-11.27,8.08L335.084,318.257z" fill="#37404D" />
            </svg>
        </div>
        <div class="features">
            <ul>
                <li><span>SELECT aircraft_code, model FROM aircrafts_data;</span> Edits</li>
                <li><span>1</span> 1</li>
                <li><span>Recent</span> SELECT airport_code, airport_name FROM airports_data;</li>
                <li><span>X</span> SELECT * FROM flights;</li>
            </ul>
        </div>
        <a href="#" class="btn">SELECT</a>
    </div>
    """,
    unsafe_allow_html=True
)

def get_sql(query):
    input_text = "translate English to SQL: %s </s>" % query
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                             attention_mask=features['attention_mask'])
  
    return tokenizer.decode(output[0])

queries = [
    ("Find bookings made on a specific date", "SELECT book_ref, total_amount FROM bookings WHERE book_date = '[specific_date]';"),
    ("Select all aircrafts and their models", "SELECT aircraft_code, model FROM aircrafts_data;"),
    ("Select airports with their names and cities", "SELECT airport_code, airport_name->>'name' AS airport_name, city->>'name' AS city FROM airports_data;"),
    # Add more queries as needed
]

def search(searchterm: str) -> List[str]:
    # Filter queries based on searchterm
    result = [query[0] for query in queries if searchterm.lower() in query[0].lower()]
    return result

# Streamlit app with custom search box
def main():
    user_query = st.text_input("Enter your search query:")

    ai_result = get_sql(user_query)
    ai_result = ai_result.replace('<pad>', '').replace('</s>', ';').strip()
    st.subheader("AI Results:")
    st.text(ai_result)

    if st.button("Execute SQL"):
        cursor = conn.cursor()
        sql_result = cursor.execute(user_query).fetchall()
        st.subheader("SQL Query Results:")
        st.write(sql_result)
    

    # CSS styles
    css_styles2 = """
    <style>
        .ui {
            display: flex;
            justify-content: space-around;
            list-style: none;
            padding: 0;
        }

        .card {
            background: #BAD2DE;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.4);
            border-radius: 5px;
            margin: 20px;
            padding: 20px;
            text-align: center;
            color: black;
            width: calc(33.3333% - 40px);
        }

        .card p {
            margin: 10px 0;
            font-size: 14px;
        }
    </style>
    """

    # Display CSS styles
    st.markdown(css_styles2, unsafe_allow_html=True)

    # Sample data
    if st.button("Test"):
        cursor = conn.cursor()
        sql_test = cursor.execute("SELECT aircraft_code, model FROM aircrafts_data;").fetchall()

        st.subheader("SQL Test")

        # Display SQL test results in the card structure
        st.markdown("<ul class='ui'>", unsafe_allow_html=True)
        for row in sql_test:
            st.markdown(f"""
                <li class='card'>
                    {''.join([f"<p>{item}</p>" for item in row])}
                </li>
            """, unsafe_allow_html=True)
        st.markdown("</ul>", unsafe_allow_html=True)

        conn.close()
        
table_scorecard = """
<div class="ui five small statistics">
  <div class="grey statistic">
    <div class="value">"""+str(df[df['TABLE_TYPE'] == 'BASE TABLE']['TABLE_ID'].count())+"""
    </div>
    <div class="grey label">
      Tables
    </div>
  </div>
    <div class="grey statistic">
        <div class="value">"""+str(df[df['TABLE_TYPE'] == 'VIEW']['TABLE_ID'].count())+"""
        </div>
        <div class="label">
        Views
        </div>
    </div>
    <div class="grey statistic">
        <div class="value">"""+str(df[df['TABLE_TYPE'] == 'MATERIALIZED VIEW']['TABLE_ID'].count())+"""
        </div>
        <div class="label">
        Materialized Views
        </div>
    </div>    
  <div class="grey statistic">
    <div class="value">
      """+human_format(df['ROW_COUNT'].sum())+"""
    </div>
    <div class="label">
      Rows
    </div>
  </div>

  <div class="grey statistic">
    <div class="value">
      """+human_bytes(df['BYTES'].sum())+" "+human_bytes_text(df['BYTES'].sum())+"""
    </div>
    <div class="label">
      Data Size
    </div>
  </div>
</div>"""

table_scorecard += """<br><br><br><div id="mydiv" class="ui centered cards">"""


for index, row in df.iterrows():
    table_scorecard += """
<div class="card"">   
    <div class=" content """+header_bg(row['TABLE_TYPE'])+"""">
            <div class=" header smallheader">"""+row['TABLE_NAME']+"""</div>
    <div class="meta smallheader">"""+row['TABLE_CATALOG']+"."+row['TABLE_SCHEMA']+"""</div>
    </div>
    <div class="content">
        <div class="description"><br>
            <div class="column kpi number">"""+human_format(row['ROW_COUNT'])+"""<br>
                <p class="kpi text">Rows</p>
            </div>
            <div class="column kpi number">"""+human_bytes(row['BYTES'])+"""<br>
                <p class="kpi text">"""+human_bytes_text(row['BYTES'])+"""</p>
            </div>
            <div class="column kpi number">"""+"{0:}".format(row['COLUMN_COUNT'])+"""<br>
                <p class="kpi text">Columns</b>
            </div>
        </div>
    </div>
    <div class="extra content">
        <div class="meta"><i class="table icon"></i> Table Type: """+(row['TABLE_TYPE'])+"""</div>
        <div class="meta"><i class="user icon"></i> Owner: """+str(row['TABLE_OWNER'])+""" </div>
        <div class="meta"><i class="calendar alternate outline icon"></i> Created On: """+(row['CREATED'].strftime("%Y-%m-%d"))+"""</div>
    </div>
    <div class="extra content" """+view_details+"""> 
        <div class="meta"><i class="history icon"></i> Time Travel: """+str((row['RETENTION_TIME'])).strip(".0")+"""</div>
        <div class="meta"><i class="edit icon"></i> Last Altered: """+(row['LAST_ALTERED'].strftime("%Y-%m-%d"))+"""</div>
        <div class="meta"><i class="calendar times outline icon"></i> Transient: """+str(row['IS_TRANSIENT'])+""" </div>
        <div class="meta"><i class="th icon"></i> Auto Clustering: """+str(row['AUTO_CLUSTERING_ON'])+""" </div>
        <div class="meta"><i class="key icon"></i> Clustering Key: """+str(row['IS_TRANSIENT'])+""" </div>
        <div class="meta"><i class="comment alternate outline icon"></i> Comment: """+str(row['IS_TRANSIENT'])+""" </div>
    </div>
</div>"""

st.markdown(table_scorecard, unsafe_allow_html=True)        

if __name__ == '__main__':
    main()

In [ ]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

In [ ]:
#Test to take ai result and edit it

"""def process_and_execute_sql(sql_query, database_path):
    # Remove <pad> and </s> from the SQL query
    processed_query = sql_query.replace('<pad>', '').replace('</s>', '').strip()

    # Add a semicolon at the end of the query
    if not processed_query.endswith(';'):
        processed_query += ';'

    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)

    try:
        # Execute the processed SQL query
        cursor = conn.cursor()
        cursor.execute(processed_query)

        # Fetch the results (if any)
        result = cursor.fetchall()

        return result

    except sqlite3.Error as e:
        return f"Error executing SQL query: {e}"

    finally:
        # Close the database connection
        conn.close()

# Example usage
ai_generated_query = "<pad> SELECT COUNT Name FROM table</s>"
database_path = '/path/to/your/database.sqlite'

result = process_and_execute_sql(ai_generated_query, database_path)

# Display the result
print(result)"""

1. take sql_result and put into a dataframe, with labels 
2. Auto complete for a tuple function

In [ ]:
def search(searchterm: str) -> List[str]:
    result = ["Find bookings made on friday" =  "SELECT book_ref, total_amount FROM bookings WHERE book_date = '[specific_date]';
"]
              
    return result